This is a list of antimicrobial medicines. 

In [1]:
from ebmdatalab import bq
import os
import pandas as pd
import numpy as np

#mapping = pd.read_csv('../lib/ingredients.csv')
#cats = pd.read_csv('../lib/categories.csv', header = None, names=["category_code","category"])

#mapping.groupby('product_id')["category_id"].nunique()
#mapping['product_id'].nunique()

#mapping = mapping.join(cats, on='product_id', how="left")



## Antimicrobials in BNF

In [2]:
sql = '''
WITH bnf_codes AS (  
  SELECT presentation_code as bnf_code, 
  IF(para IN ('Some other antibacterials','Antituberculosis drugs', 'Antileprotic drugs'),
        'Others', para) AS para  
  FROM hscic.bnf 
  WHERE 
  (presentation_code LIKE '0501%')
) 
  
SELECT "vmp" AS type, vmp.id, vmp.bnf_code, vmp.nm, ing.nm AS ingredient, 
  route.descr as route, para as paragraph
FROM dmd.vmp 
INNER JOIN bnf_codes b ON b.bnf_code = vmp.bnf_code
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing
LEFT JOIN dmd.droute on vmp.id = droute.vmp	
LEFT JOIN dmd.route on route.cd = droute.route

ORDER BY type, nm  '''

antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data','antibac_meds.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
antibac_meds[["id", "bnf_code"]].count()

antibac_meds["source"] = "bnf"

In [3]:
#antibac_meds.merge(mapping, left_on='id', right_on='product_id', how='left')
# check that each ingredient is mapped to just one paragraph
print(antibac_meds.groupby(['ingredient'])['paragraph'].nunique().max())
print("No. of paras: ", antibac_meds['paragraph'].nunique())
para_lookup = antibac_meds[['ingredient','paragraph']].drop_duplicates()

1
No. of paras:  11


## Dm+d Additions
There are some medicines used in hospital only, which don't have BNF codes (used for primary care). 

We will manually select these from dmd using the ingredients found in the products we identified from BNF codes.

In [4]:
  
# set name of codelist for exporting file
codelist_name="antibac_meds"

# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = antibac_meds["ingredient"].drop_duplicates()
print("No of antibiotic ingredients from bnf list: ", len(names))

# filter out ingredients identified as not being antimicrobial agents
names_filtered = [k for k in names if k not in ["Citric acid", "Thalidomide", "Sodium bicarbonate", "Sodium citrate"]]
print("No of antibiotic ingredients after filtering: ", len(names_filtered))

# convert list to tuple for use in SQL query
names_tuple = tuple(names_filtered)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
WITH dmd_codes as (
SELECT vmp.id, vmp.vpidprev, bnf_code, vmp.nm, ing.nm AS ingredient, 
    route.descr as route, ddd.ddd, ddd.ddd_uomcd AS ddd_uomcd, who.category
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.droute on vmp.id = droute.vmp  -- NB this introduces some duplicates we will remove later
LEFT JOIN dmd.route on route.cd = droute.route
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid
LEFT JOIN jonm.who_aware_vmp who on vmp.id = who.vmp)

-- Put all old and new IDs together in single column
SELECT "vmp" AS type, id, bnf_code, 
nm, ingredient, route, ddd, ddd_uomcd, category
FROM dmd_codes

UNION ALL 

SELECT "vmp" AS type, vpidprev AS id, bnf_code, 
nm, ingredient, route, ddd, ddd_uomcd, category
FROM dmd_codes
WHERE vpidprev IS NOT NULL

ORDER BY type, nm
     
 '''

dmd_antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'dmd_{codelist_name}.csv'))

print("No of antibiotic ingredients in dmd list: ", dmd_antibac_meds["ingredient"].nunique())

No of antibiotic ingredients from bnf list:  122
No of antibiotic ingredients after filtering:  118
No of antibiotic ingredients in dmd list:  118


**Note the above should have captured all the codes we previously found by selecting based on BNF codes so we don't need to join the BNF + DMD tables.**

In [6]:
## NB - the ID column contains multiple codes for the same drug names.

test = dmd_antibac_meds.copy()
test["count"] = test.groupby("nm")["id"].transform('nunique')
print("n drugs with more than one vpid code:", test.loc[test["count"]>1]["nm"].nunique())

n drugs with more than one vpid code: 132


In [8]:
# Address multiple WHO categories and missing ones

dmd_antibac_meds_2 = dmd_antibac_meds.copy()
dmd_antibac_meds_2["cat_count"] = dmd_antibac_meds_2.groupby("id")["category"].transform('nunique')
print("Max no of categories per vmp:", dmd_antibac_meds_2["cat_count"].max()) # this should be 2 but not 3.
# Since there is currently a max of 2 diff values we can compare max and min and pick one.
dmd_antibac_meds_2["cat_1"] = dmd_antibac_meds_2.groupby("id")["category"].transform('min').astype(str)
dmd_antibac_meds_2["cat_2"] = dmd_antibac_meds_2.groupby("id")["category"].transform('max').astype(str)

# Access, Watch, Reserve: possible pair combinations in alphabetical order
# A + W = Watch
# A + R = Reserve
# R + W = Reserve
s = dmd_antibac_meds_2["category"].copy()
s.mask((dmd_antibac_meds_2["cat_count"]>1) & (dmd_antibac_meds_2["cat_1"]=="Reserve"), dmd_antibac_meds_2["cat_1"], inplace=True)
s.mask((dmd_antibac_meds_2["cat_count"]>1) & (dmd_antibac_meds_2["cat_1"]=="Access"), dmd_antibac_meds_2["cat_2"], inplace=True)
# fill resolved category and fill missing with "Uncategorised". (Also strip as there were two versions of "Watch")
dmd_antibac_meds_2["category"] = s.str.strip().fillna("Uncategorised")
dmd_antibac_meds_2 = dmd_antibac_meds_2.drop(["cat_count", "cat_1", "cat_2"], axis=1)

# recheck count
dmd_antibac_meds_2["cat_count"] = dmd_antibac_meds_2.groupby("id")["category"].transform('nunique')
print("Max no of categories per vmp after resolving:", dmd_antibac_meds_2["cat_count"].max()) # this should now be 1
dmd_antibac_meds_2 = dmd_antibac_meds_2.drop(["cat_count"], axis=1)
print(dmd_antibac_meds_2.groupby(["category"])["nm"].nunique()) 
#dmd_antibac_meds_2.loc[pd.isnull(dmd_antibac_meds_2["category"])].sort_values(by="ingredient")#.to_csv(os.path.join('..','data',f'abx_without_who_category.csv'))


Max no of categories per vmp: 2
Max no of categories per vmp after resolving: 1
category
Access           324
Reserve           33
Uncategorised    221
Watch            319
Name: nm, dtype: int64


### BNF Paragraphs

In [9]:
# join BNF paragraph via ingredients
print("No of records before join:", len(dmd_antibac_meds_2["id"]))
dmd_antibac_meds_3 = dmd_antibac_meds_2.merge(para_lookup, on='ingredient', how='left')
print("No of records after join:",len(dmd_antibac_meds_3["id"]))

# some (3) multi-ingredient products map to different paragraphs (Genta + Clinda x2, Genta + Vancomycin). 
# We'll assign them to Other.
dmd_antibac_meds_3["para_count"] = dmd_antibac_meds_3.groupby("id")["paragraph"].transform('nunique')
#print(dmd_antibac_meds_3.loc[dmd_antibac_meds_3["para_count"]>1])
dmd_antibac_meds_3.loc[dmd_antibac_meds_3["para_count"]>1, "paragraph"] =  "Other"

dmd_antibac_meds_3.head()

No of records before join: 1251
No of records after join: 1251


,type,id,bnf_code,nm,ingredient,route,ddd,ddd_uomcd,category,paragraph,para_count
0,vmp,35899811000001104,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intramuscular,1.0,258682000.0,Access,Aminoglycosides,1
1,vmp,35899811000001104,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intravenous,1.0,258682000.0,Access,Aminoglycosides,1
2,vmp,324143009,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intramuscular,1.0,258682000.0,Access,Aminoglycosides,1
3,vmp,324143009,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intravenous,1.0,258682000.0,Access,Aminoglycosides,1
4,vmp,33516711000001107,NaN,Amikacin 2.5% eye drops preservative free,Amikacin sulfate,Ocular,NaN,NaN,Access,Aminoglycosides,1


### Investigate routes of administration

In [10]:
dmd_antibac_meds_3.groupby("route")["nm"].nunique().sort_values(ascending=False)

route
Oral                                      544
Intravenous                               176
Intramuscular                              67
Ocular                                     53
Cutaneous                                  43
Route of administration not applicable     28
Auricular                                  19
Inhalation                                 17
Intrathecal                                 7
Intraarticular                              5
Intracameral                                5
Intrapleural                                5
Intravitreal                                4
Intralesional                               3
Vaginal                                     3
Rectal                                      3
Intraperitoneal                             3
Nasal                                       2
Oromucosal                                  2
Gingival                                    2
Gastroenteral                               2
Subcutaneous                

In [11]:
# categorise routes of administration

condlist = [dmd_antibac_meds_3["route"]== "Oral",
            dmd_antibac_meds_3["route"].isin(["Intravenous", "Subcutaneous", "Intramuscular", "Intrapleural", 
                                        "Intraperitoneal", "Intraarticular", "Intracavernous", "Intrathecal",
                                        "Intracerebroventricular"]),
            dmd_antibac_meds_3["route"].isin(["Cutaneous", "Auricular","Vaginal","Intralesional",
                                        "Ocular", "Nasal", "Oromucosal", "Gingival", "Dental"]),
            dmd_antibac_meds_3["route"].isin(["Inhalation", "Intravesical", "Gastroenteral"])
                                        ]
choicelist = ["Oral", "Injectable", "Topical", "Other"]
dmd_antibac_meds_3["Route"] = np.select(condlist, choicelist, default="Other")

# check for and correct vmps with multiple routes
dmd_antibac_meds_3 = dmd_antibac_meds_3.copy()
dmd_antibac_meds_3["Route_count"] = dmd_antibac_meds_3.groupby(["id"])["Route"].transform('nunique')
# most of those with mixed routes have "for injection" or "for infusion" in their name
dmd_antibac_meds_3.loc[(dmd_antibac_meds_3["Route_count"]>1) & (
                        (dmd_antibac_meds_3["nm"].str.contains("injection")) | (
                                    dmd_antibac_meds_3["nm"].str.contains("infusion"))), 
                                    "Route"] =  "Injectable"
#recount and assign any remaining to "Other" - this should be restricted to colistin                              
dmd_antibac_meds_3["Route_count"] = dmd_antibac_meds_3.groupby(["id"])["Route"].transform('nunique')
dmd_antibac_meds_3.loc[dmd_antibac_meds_3["Route_count"]>1, "Route"] =  "Other"

print(dmd_antibac_meds_3.loc[dmd_antibac_meds_3["Route_count"]>1][["id", "nm", "Route_count", "route", "Route"]])

dmd_antibac_meds_3 = dmd_antibac_meds_3.drop("route", axis=1)

                   id                                      nm  Route_count  \
399  4615111000001106  Colistin 19.5million unit powder vials            2   
400  4615111000001106  Colistin 19.5million unit powder vials            2   
401  4615111000001106  Colistin 19.5million unit powder vials            2   
402  4615111000001106  Colistin 19.5million unit powder vials            2   

         route  Route  
399       Oral  Other  
400  Auricular  Other  
401  Cutaneous  Other  
402     Ocular  Other  


In [12]:
# check for remaining dmds with multiple routes of administration
test = dmd_antibac_meds_3.copy()#[["id", "bnf_code","Route"]]
test["Route_count"] = test.groupby(["id"])["Route"].transform('nunique')

test = test.loc[test["Route_count"]>1]
print(test)

Empty DataFrame
Columns: [type, id, bnf_code, nm, ingredient, ddd, ddd_uomcd, category, paragraph, para_count, Route, Route_count]
Index: []


In [13]:
# check number of distinct medicines
print("Unique VMP names:", dmd_antibac_meds_3["nm"].agg({"nunique"})[0])
print("VMP name-code combinations:", dmd_antibac_meds_3.groupby("nm")["id"].agg({"nunique"}).sum()[0])
print("VMP name-ingredient combinations:", dmd_antibac_meds_3.groupby("nm")["ingredient"].agg({"nunique"}).sum()[0])

Unique VMP names: 897
VMP name-code combinations: 1029
VMP name-ingredient combinations: 942


In [14]:
# check how many products have DDDs
dmd_antibac_meds_3["ddd_flag"] = np.where(dmd_antibac_meds_3["ddd"].notnull(), 1, 0)
print(dmd_antibac_meds_3["ddd_flag"].nunique())

print("Max DDDs per drug:\n", dmd_antibac_meds_3.groupby(["id"])["ddd"].agg({"nunique","count"}).max())
print("No of drugs with and without DDD by route\n:", dmd_antibac_meds_3.groupby(["Route","ddd_flag"])["nm"].nunique().unstack())


2
Max DDDs per drug:
 nunique    1
count      6
dtype: int64
No of drugs with and without DDD by route
: ddd_flag      0    1
Route               
Injectable   71  118
Oral         40  499
Other        41   11
Topical     115    2


In [15]:
dmd_antibac_meds_3 = dmd_antibac_meds_3.drop(["Route_count","para_count"], axis=1)

In [16]:
dmd_antibac_meds_3.to_csv(os.path.join('..','data','antibac_codelist.csv')) #export to csv here
